In [1]:
import pandas as pd
import os
import sys
sys.path.append('../../src')
from utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL
from data_process import data_split, neg_sample
import datetime
import numpy as np

## Loading the application 

In [2]:
apps = pd.read_csv('../../data/jobs/tsv/apps.tsv', sep='\t', encoding='utf-8')
apps.rename(columns = {'UserID':DEFAULT_USER_COL, 'ApplicationDate':DEFAULT_TIMESTAMP_COL, \
                      'JobID':DEFAULT_ITEM_COL
                      }, inplace=True)

In [3]:
apps.columns

Index(['userid', 'WindowID', 'Split', 'timestamp', 'itemid'], dtype='object')

## Loading the jobs info

all columns: ['JobID', 'WindowID', 'Title', 'Description', 'Requirements','City', 'State', 'Country', 'Zip5', 'StartDate','EndDate']

selected_columns = ['JobID', 'WindowID','City_job', 'State_job', 'Country_job', 'Zip5', 'StartDate','EndDate']

In [5]:
selected_feature = ['JobID', 'WindowID','City', 'State', 'Country', 'Zip5', 'StartDate','EndDate']
jobs = pd.read_csv('../../data/jobs/tsv/jobs.tsv', sep='\t', encoding='utf-8', on_bad_lines='skip', usecols = selected_feature)

jobs.rename(columns = {'JobID':DEFAULT_ITEM_COL, 'City':'City_job', \
                       'State':'State_job', 'Country':'Country_job', \
                      'Zip5':'Zip_job'}, inplace=True)

/tmp/ipykernel_159619/430080897.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv('../../data/jobs/tsv/jobs.tsv', sep='\t', encoding='utf-8', on_bad_lines='skip', usecols = selected_feature)


In [6]:
jobs.columns

Index(['itemid', 'WindowID', 'City_job', 'State_job', 'Country_job', 'Zip_job',
       'StartDate', 'EndDate'],
      dtype='object')

## Loading the user history info

full columns: Index(['UserID', 'WindowID', 'Split', 'Sequence', 'JobTitle'], dtype='object')

selected columns:['UserID', 'WindowID', 'Split', 'Sequence']

In [7]:
selected_col=['UserID', 'WindowID', 'Split', 'Sequence']
user_history = pd.read_csv('../../data/jobs/tsv/user_history.tsv', sep='\t', encoding='utf-8', usecols=selected_col)


### With jobtitle

In [8]:
# user_history.rename(columns = {'UserID':DEFAULT_USER_COL, 'JobTitle':'JobTitle_user'}, inplace=True)
# user_history = user_history[user_history['JobTitle_user'].notna()]
# user_history_title = user_history.groupby('userid')['JobTitle_user'].\
# apply(lambda x:','.join(x)).reset_index()


## Loading the user profile info

In [9]:
users = pd.read_csv('../../data/jobs/tsv/users.tsv', sep='\t', encoding='utf-8')
users.rename(columns = {'UserID':DEFAULT_USER_COL, 'ZipCode':'Zip_user'}, inplace=True)

In [10]:
users.columns

Index(['userid', 'WindowID', 'Split', 'City', 'State', 'Country', 'Zip_user',
       'DegreeType', 'Major', 'GraduationDate', 'WorkHistoryCount',
       'TotalYearsExperience', 'CurrentlyEmployed', 'ManagedOthers',
       'ManagedHowMany'],
      dtype='object')

## Merge data 

### Have job title

In [11]:
# df_merge = apps.merge(users, how='left', on=[DEFAULT_USER_COL]) \
# .merge(user_history_title, how='left', on=[DEFAULT_USER_COL]) \
# .merge(jobs, how='left', on=[DEFAULT_ITEM_COL])

### Don't have job title

In [12]:
df_merge = apps.merge(users, how='left', on=[DEFAULT_USER_COL]) \
.merge(jobs, how='left', on=[DEFAULT_ITEM_COL])

In [13]:
df_merge = df_merge.drop(['WindowID_x', 'Split_x'], axis=1).rename(columns = {'WindowID_y':'WindowID_user', \
                                                            'Split_y':'Split', 'Title':'JobTitle', \
                                                            'WindowID':'WindowID_job'
                                                            })

In [14]:
df_merge.columns

Index(['userid', 'timestamp', 'itemid', 'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany', 'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate'],
      dtype='object')

## Save data

In [16]:
df_merge = df_merge[df_merge['Country']=='US']
df_merge[DEFAULT_RATING_COL] = '1'

In [18]:
df_merge.shape

(1594565, 25)

In [24]:
df_merge.to_csv("../../data/jobs/merged_all.csv", index=False)

In [25]:
# aa.to_hdf("../data/merged_apps.h5", key='all', format='table', data_columns=True, mode='w')


## Convert the datatype 

In [26]:
# cat_features = ['userid', 'itemid', 'WindowID_user', 'Split', 'City', \
#                 'State', 'Country', 'Zip_user', 'DegreeType', 'Major', \
#                'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed', \
#                'ManagedOthers', 'ManagedHowMany','WindowID_job','City_job', \
#                'State_job', 'Country_job', 'Zip_job']
# # num_features = []
# time_features = ['timestamp', 'GraduationDate','StartDate', 'EndDate' ]
# # str_features = ['JobTitle_user', 'JobTitle','Description','Requirements']

In [28]:
# # for feature_type in all_features:
# for f in cat_features:
# #     aa[f] = aa[f].fillna('none').astype('category')
# #     pd.Categorical(aa[f])
#     df_merge[f] = df_merge[f].astype('str')
# for f in time_features:
#     df_merge[f] = pd.to_datetime(df_merge[f])
# # for f in str_features:
# #     aa[f] = aa[f].astype(str)

## Subsample dataset 
Keep only US dataset and subsmaple 3-window users 
all the columns: Index(['userid', 'timestamp', 'itemid', 'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany', 'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate'],
      dtype='object')

In [42]:
# df_all = pd.read_hdf('../data/merged_apps.h5', key='all', columns=['userid', 'itemid', \
#                                                                    'WindowID_user','Country'])

In [21]:
# df_all = pd.read_hdf('../data/merged_apps.h5', key='all')
df_all = pd.read_csv("../../data/jobs/merged_all_clean.csv")

In [22]:
df_all.isnull().sum()

userid                  0
timestamp               0
itemid                  0
WindowID_user           0
Split                   0
City                    0
State                   0
Country                 0
Zip_user                0
DegreeType              0
Major                   0
GraduationDate          0
WorkHistoryCount        0
TotalYearsExperience    0
CurrentlyEmployed       0
ManagedOthers           0
ManagedHowMany          0
WindowID_job            0
City_job                0
State_job               0
Country_job             0
Zip_job                 0
StartDate               0
EndDate                 0
rating                  0
dtype: int64

Subsample certain windows dataset 

In [23]:
select_win = [3, 4, 5]

In [24]:
df_win = df_all[df_all.WindowID_user.isin(select_win)]

In [25]:
df_win.isnull().sum()

userid                  0
timestamp               0
itemid                  0
WindowID_user           0
Split                   0
City                    0
State                   0
Country                 0
Zip_user                0
DegreeType              0
Major                   0
GraduationDate          0
WorkHistoryCount        0
TotalYearsExperience    0
CurrentlyEmployed       0
ManagedOthers           0
ManagedHowMany          0
WindowID_job            0
City_job                0
State_job               0
Country_job             0
Zip_job                 0
StartDate               0
EndDate                 0
rating                  0
dtype: int64

In [26]:
df_win.to_csv("../../data/jobs/merged_sub_clean.csv", index=False)

# Crearte the leave one out data

In [10]:
path_read = "../../data/jobs/merged_sub_clean.csv"
path_save1 = "../../data/jobs/leave_one_train.csv"
path_save2 = "../../data/jobs/leave_one_test.csv"
data_split.main(path_read, path_save1, path_save2)

# Create the negative sample dataset 

### Create for the training set 

In [11]:
# path_read_test = '../../Data/jobs/leave_one_test.csv'
path_read_all = '../../data/jobs/merged_sub.csv'
path_read_train = '../../data/jobs/leave_one_train.csv'
path_save = '../../data/jobs/leave_one_train_neg.csv'
ng_num = 4
neg_sample.main(path_read_all = path_read_all,
                path_read_goal = path_read_train, 
                path_save=path_save, ng_num=ng_num, 
                path_read_neg=None, test=False, )

### Create for the test set 

In [14]:
path_read_test = '../../data/jobs/leave_one_test.csv'
path_read_all = '../../data/jobs/merged_sub_clean.csv'
path_save = '../../data/jobs/test_pos_neg'
train_neg = '../../data/jobs/leave_one_train_neg'
ng_num = 100
neg_sample.main(path_read_all=path_read_all, 
                path_read_goal=path_read_test, 
                path_save=path_save, 
                ng_num=ng_num, 
                test=True,
                path_read_neg = train_neg)

100%|█████████████████████████████████████| 74091/74091 [27:45<00:00, 44.48it/s]


# Check the correctness of the data

# Processing the missing value and datatype

In [19]:
df_all = pd.read_csv('../../data/jobs/merged_all_clean.csv')

In [20]:
df_all.isnull().sum()

userid                  0
timestamp               0
itemid                  0
WindowID_user           0
Split                   0
City                    0
State                   0
Country                 0
Zip_user                0
DegreeType              0
Major                   0
GraduationDate          0
WorkHistoryCount        0
TotalYearsExperience    0
CurrentlyEmployed       0
ManagedOthers           0
ManagedHowMany          0
WindowID_job            0
City_job                0
State_job               0
Country_job             0
Zip_job                 0
StartDate               0
EndDate                 0
rating                  0
dtype: int64

In [16]:
df_all['GraduationDate'] = df_all['GraduationDate'].fillna(pd.Timestamp('1677-09-22T12'))
df_all['TotalYearsExperience'] = df_all['TotalYearsExperience'].fillna(-1)
df_all['CurrentlyEmployed'] = df_all['CurrentlyEmployed'].fillna('MISSING')
df_all['WindowID_job'] = df_all['WindowID_job'].fillna(-1)
df_all['City_job'] = df_all['City_job'].fillna("MISSING")
df_all['State_job'] = df_all['State_job'].fillna("MISSING")
df_all['Major'] = df_all['Major'].fillna("MISSING")
df_all['Country_job'] = df_all['Country_job'].fillna("US")
df_all['StartDate'] = df_all['StartDate'].fillna(pd.Timestamp('1677-09-22T12'))
df_all['EndDate'] = df_all['EndDate'].fillna(pd.Timestamp('1677-09-22T12'))
# df_all['City'] = df_all['City'].str.lower()
# df_all['City'] = df_all['City'].str.replace(r'[^\w\s]+', '', regex=True)
# df_all['City_job'] = df_all['City_job'].str.lower()
# df_all['City_job'] = df_all['City_job'].str.replace(r'[^\w\s]+', '', regex=True)

In [17]:
df_all.isnull().sum()

userid                  0
timestamp               0
itemid                  0
WindowID_user           0
Split                   0
City                    0
State                   0
Country                 0
Zip_user                0
DegreeType              0
Major                   0
GraduationDate          0
WorkHistoryCount        0
TotalYearsExperience    0
CurrentlyEmployed       0
ManagedOthers           0
ManagedHowMany          0
WindowID_job            0
City_job                0
State_job               0
Country_job             0
Zip_job                 0
StartDate               0
EndDate                 0
rating                  0
dtype: int64

## Get the city-zip dictionary 

In [9]:
df_geo_user = df_all[['City', 'Zip_user']].rename(columns={'Zip_user':'zip'})
df_geo_job = df_all[['City_job', 'Zip_job']].rename(columns={'City_job':'City', 'Zip_job':'zip'})

In [10]:
df_geo = pd.concat([df_geo_user, df_geo_job], axis=0).dropna().reset_index(drop=True)

In [11]:
# df_geo['City'] = df_geo['City'].astype(str)
df_geo['zip'] = df_geo['zip'].astype(str)

In [12]:
df_geo = df_geo[~(df_geo['zip']=='     ')]

In [13]:
def zip_process(data, zip_name):
    zip_ser = data[zip_name].apply(lambda x: x[:5])
    zip_ser =zip_ser.apply(lambda x: '0'+ x[:4] if x[-1] == '.' else x)
    zip_ser =zip_ser.apply(lambda x: '00'+ x[:3] if x[-2] == '.' else x)
    zip_ser =zip_ser.apply(lambda x: x[:4]+'0' if x[-1] == '-' else x)
    zip_ser =zip_ser.apply(lambda x: '0'*(5-len(x))+x if len(x) <5  else x)
    return zip_ser.astype(str)

In [14]:
zip_ser = zip_process(df_geo, 'zip')
df_geo['zip_new'] = zip_ser

In [15]:
zip_dic= dict(zip(df_geo.City, df_geo.zip_new))

In [16]:
df_all['Zip_user'] = df_all['Zip_user'].fillna(value=zip_dic)
df_all['Zip_job'] = df_all['Zip_job'].fillna(value=zip_dic)

In [17]:
df_all['Zip_user'] = df_all['Zip_user'].astype(str)
df_all['Zip_job'] = df_all['Zip_job'].astype(str)

In [18]:
df_all['Zip_user'] = zip_process(df_all, 'Zip_user')
df_all['Zip_job'] = zip_process(df_all, 'Zip_job')

In [19]:
sum(df_all['Zip_user'].apply(lambda x: True if len(x)!=5 else False))

0

In [20]:
sum(df_all['Zip_job'].apply(lambda x: True if len(x)!=5 else False))

0

In [21]:
df_all.isnull().sum()

userid                  0
timestamp               0
itemid                  0
WindowID_user           0
Split                   0
City                    0
State                   0
Country                 0
Zip_user                0
DegreeType              0
Major                   0
GraduationDate          0
WorkHistoryCount        0
TotalYearsExperience    0
CurrentlyEmployed       0
ManagedOthers           0
ManagedHowMany          0
WindowID_job            0
City_job                0
State_job               0
Country_job             0
Zip_job                 0
StartDate               0
EndDate                 0
rating                  0
dtype: int64

## Convert the datatype 

In [22]:
cat_features = ['userid', 'itemid', 'WindowID_user', 'Split', 'City', \
                'State', 'Country', 'Zip_user', 'DegreeType', 'Major', \
               'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed', \
               'ManagedOthers', 'ManagedHowMany','WindowID_job','City_job', \
               'State_job', 'Country_job', 'Zip_job']
# num_features = []
time_features = ['timestamp', 'GraduationDate','StartDate', 'EndDate' ]
# str_features = ['JobTitle_user', 'JobTitle','Description','Requirements']

In [23]:
# for feature_type in all_features:
for f in cat_features:
    df_all[f] = df_all[f].astype('category')
for f in time_features:
    df_all[f] = pd.to_datetime(df_all[f])
# for f in str_features:
#     aa[f] = aa[f].astype(str)

In [24]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1594565 entries, 0 to 1594564
Data columns (total 25 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   userid                1594565 non-null  category      
 1   timestamp             1594565 non-null  datetime64[ns]
 2   itemid                1594565 non-null  category      
 3   WindowID_user         1594565 non-null  category      
 4   Split                 1594565 non-null  category      
 5   City                  1594565 non-null  category      
 6   State                 1594565 non-null  category      
 7   Country               1594565 non-null  category      
 8   Zip_user              1594565 non-null  category      
 9   DegreeType            1594565 non-null  category      
 10  Major                 1594565 non-null  category      
 11  GraduationDate        1594565 non-null  datetime64[ns]
 12  WorkHistoryCount      1594565 non-null  ca

In [18]:
df_all.to_csv('../../data/jobs/merged_all_clean.csv', index=False)